# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [2]:
crowd_data_orig = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data_orig.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_36932\1640917472.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data_orig = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


Keep only data with a crowd size, information about what the crowd gathered for, and a FIPS code.

In [3]:
crowd_data = crowd_data_orig[(crowd_data_orig["issues"].notnull()) & 
                             (crowd_data_orig["size_mean"].notnull()) & 
                             (crowd_data_orig["fips_code"].notnull())]
crowd_data.isna().sum()

date                    0
locality                1
state                   9
location_detail      2775
online                 10
                     ... 
lon                     3
resolved_locality     347
resolved_county      1930
resolved_state         11
fips_code               0
Length: 62, dtype: int64

In [4]:
# source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
# crowd_data.drop(columns=source_cols, inplace=True)
# crowd_data.head(5)
# crowd_data["fips_code"] = crowd_data["fips_code"].astype(np.int16)
# crowd_data["fips_code"]

crowd_data["year"] = crowd_data["date"].apply(lambda x:x[:4]).astype(int)
crowd_data["year"]


C:\Users\alexa\AppData\Local\Temp\ipykernel_36932\3801195337.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crowd_data["year"] = crowd_data["date"].apply(lambda x:x[:4]).astype(int)


2        2017
8        2017
13       2017
14       2017
16       2017
         ... 
72155    2020
72156    2020
72167    2020
72168    2020
72172    2020
Name: year, Length: 33821, dtype: int64

Limit to the years 2017 and 2018

In [5]:
crowd_data = crowd_data[(crowd_data["year"] == 2017) | (crowd_data["year"] == 2018)]

In [6]:
print(len(crowd_data[crowd_data["fips_code"].isna()]), "out of", len(crowd_data))

0 out of 16768


0 unresolved localities out of 16768; we need not discard any data.

In [7]:
# crowd_data = crowd_data[crowd_data["fips_code"].notna()]
# crowd_data["fips_code"] = crowd_data["fips_code"].astype(int)
# crowd_data["fips_code"]

### American Community Survey

In [8]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

def regex_filter(string, myregex):
    if string:
        mo = re.search(myregex, string)
        if mo:
            return True
        else:
            return False
    else:
        return False

census_vars_orig = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars_orig[(census_vars_orig["GROUP"] != "N/A") & (census_vars_orig["GROUP"] != "DP02PR")]
census_vars_pcts = census_vars[census_vars["VARIABLE"].apply(regex_filter,myregex=r"PE$")]
census_vars_pcts

,YEAR,DATASET,GROUP,VARIABLE,LABEL,SUGGESTED_WEIGHT,VALUES
315,2017,acs/acs1/profile,DP02,DP02_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households,NaN,None
317,2017,acs/acs1/profile,DP02,DP02_0002PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
319,2017,acs/acs1/profile,DP02,DP02_0003PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
321,2017,acs/acs1/profile,DP02,DP02_0004PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
323,2017,acs/acs1/profile,DP02,DP02_0005PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
...,...,...,...,...,...,...,...
1347,2017,acs/acs1/profile,DP05,DP05_0085PE,Percent!!HISPANIC OR LATINO AND RACE!!Total po...,NaN,None
1349,2017,acs/acs1/profile,DP05,DP05_0086PE,Percent!!Total housing units,NaN,None
1351,2017,acs/acs1/profile,DP05,DP05_0087PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None
1353,2017,acs/acs1/profile,DP05,DP05_0088PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None


Solely request percents. The list of variables includes what should be the total population as a percent—`DP05_0001P`—but, as that would always be 100%, the wise people at the US Census Bureau made such equal to `DP05_0001E`, the total population estimate, necessitating no further adjustment.

`DP02PR` contains the same data as `DP02`, but solely contains percents; as we are already limiting to percents, there is no reason to include both.

In [9]:
ced.variables.group_tree(ACS1_PROFILE, 2017,"DP02")

+ Estimate
    + HOUSEHOLDS BY TYPE
        + Total households (DP02_0001E)
            + Family households (families) (DP02_0002E)
                + With own children of the householder under 18 years (DP02_0003E)
                + Married-couple family (DP02_0004E)
                    + With own children of the householder under 18 years (DP02_0005E)
                + Male householder, no wife present, family (DP02_0006E)
                    + With own children of the householder under 18 years (DP02_0007E)
                + Female householder, no husband present, family (DP02_0008E)
                    + With own children of the householder under 18 years (DP02_0009E)
            + Nonfamily households (DP02_0010E)
                + Householder living alone (DP02_0011E)
                    + 65 years and over (DP02_0012E)
        + Households with one or more people under 18 years (DP02_0013E)
        + Households with one or more people 65 years and over (DP02_0014E)
        + Aver

In [10]:
census_data = [ced.download(ACS1_PROFILE, year, pd.concat([pd.Series(["NAME"]), census_vars_pcts["VARIABLE"]], ignore_index=True), 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")
               for year in range(2017,2019)]


census_data = pd.concat(census_data, keys = np.arange(2017,2019), names=["year"])
census_data

NAME  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                         
2017 44220                                                         Springfield, OH Metro Area   
     44300                                                       State College, PA Metro Area   
     44340                                                          Statesboro, GA Micro Area   
     44420                                                 Staunton-Waynesboro, VA Metro Area   
     44620                                                       Stevens Point, WI Micro Area   
...                                                                                       ...   
2018 31820                                                           Manitowoc, WI Micro Area   
     31860                                               Mankato-North Mankato, MN Metro Area   
     31900                                                           Mansfield, OH Metro Area   
     31940                                                        Marinette, WI-MI Micro Area   
     31980                                                              Marion, IN Micro Area   

                                                         DP02_0001PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                
2017 44220                                                   54026.0   
     44300                                                   58629.0   
     44340                                                   27802.0   
     44420                                                   50325.0   
     44620                                                   28679.0   
...                                                              ...   
2018 31820                                                   35259.0   
     31860                                                   39263.0   
     31900                                                   48105.0   
     31940                                                   29110.0   
     31980                                                   26923.0   

                                                         DP02_0002PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                
2017 44220                                                      64.7   
     44300                                                      56.3   
     44340                                                      60.0   
     44420                                                      67.1   
     44620                                                      57.8   
...                                                              ...   
2018 31820                                                      60.1   
     31860                                                      62.0   
     31900                                                      63.8   
     31940                                                      58.3   
     31980                                                      65.0   

                                                         DP02_0003PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                
2017 44220                                                      26.1   
     44300                                                      23.6   
     44340                                                      27.8   
     44420                                                      23.5   
     44620                                                      22.2   
...                                                              ...   
2018 31820                                                      23.1   
     31860                                                      25.8   
     31900                                                      23.0   
     31940                                                      20.4   
     31980                                                      26.8   

                

In [11]:
# metro_micro_area_text = re.compile(r"(\s(Metro|Micro)\sArea)$")
# census_data["NAME"].apply(lambda x: metro_micro_area_text.sub("",x))
# # re.sub(string="New York Metro Area", pattern=r"(Metro|Micro)\sArea$", repl="hello")

### Most Important Problem Dataset, Second Release (2024)

The aggregated data may be found here: https://williamslaro.github.io/talks/dataset2. Previously, the individual data was used, necessitating much preprocessing.

In [12]:
issue_data = pd.read_csv("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\MIPD-2024-Aggregate\\MIPDV2-All-year.csv",encoding="ANSI",
                         index_col=["year","problem"])
issue_data.head(20)

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
1939 1          1           3            8989  45.401001  MIPD (General)   
     2          1           3            8989   6.861000  MIPD (General)   
     3          1           3            8989   0.200000  MIPD (General)   
     4          1           3            8989   0.554000  MIPD (General)   
     5          1           3            8989   4.629000  MIPD (General)   
     6          1           3            8989  33.779999  MIPD (General)   
     7          1           3            8989   0.056000  MIPD (General)   
     8          1           3            8989   0.044000  MIPD (General)   
     9          1           3            8989   1.432000  MIPD (General)   
     10         1           3            8989   4.123000  MIPD (General)   
     11         1           3            8989   0.700000  MIPD (General)   
     12         1           3            8989   0.168000  MIPD (General)   
     13         1           3            8989   1.461000  MIPD (General)   
     14         1           3            8989   0.078000  MIPD (General)   
     15         1           3            8989   0.512000  MIPD (General)   
1940 1          1           1            2907  24.200001  MIPD (General)   
     2          1           1            2907   3.474000  MIPD (General)   
     3          1           1            2907   0.069000  MIPD (General)   
     4          1           1            2907   0.172000  MIPD (General)   
     5          1           1            2907  13.152000  MIPD (General)   

                                             catdesc  \
year problem                                           
1939 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
1940 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   

                                   problemname  \
year problem                                     
1939 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
1940 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   

       

Limit to relevant years

In [13]:
issue_data = issue_data.loc[[2017,2018]]
issue_data

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
2017 1          1           2            1999  19.669001  MIPD (General)   
     2          1           2            1999  16.531000  MIPD (General)   
     3          1           2            1999   4.969000  MIPD (General)   
     4          1           2            1999   1.275000  MIPD (General)   
     5          1           2            1999  15.047000  MIPD (General)   
     6          1           2            1999  11.080000  MIPD (General)   
     7          1           2            1999   0.222000  MIPD (General)   
     8          1           2            1999   2.157000  MIPD (General)   
     9          1           2            1999   5.814000  MIPD (General)   
     10         1           2            1999  14.678000  MIPD (General)   
     11         1           2            1999   0.426000  MIPD (General)   
     12         1           2            1999   0.000000  MIPD (General)   
     13         1           2            1999   3.186000  MIPD (General)   
     14         1           2            1999   0.269000  MIPD (General)   
     15         1           2            1999   4.676000  MIPD (General)   
2018 1          1           2            1983  14.319000  MIPD (General)   
     2          1           2            1983  11.224000  MIPD (General)   
     3          1           2            1983   8.407000  MIPD (General)   
     4          1           2            1983   3.457000  MIPD (General)   
     5          1           2            1983  15.009000  MIPD (General)   
     6          1           2            1983  12.685000  MIPD (General)   
     7          1           2            1983   0.282000  MIPD (General)   
     8          1           2            1983   1.959000  MIPD (General)   
     9          1           2            1983   5.000000  MIPD (General)   
     10         1           2            1983  17.754000  MIPD (General)   
     11         1           2            1983   0.707000  MIPD (General)   
     12         1           2            1983   0.000000  MIPD (General)   
     13         1           2            1983   4.345000  MIPD (General)   
     14         1           2            1983   0.125000  MIPD (General)   
     15         1           2            1983   4.728000  MIPD (General)   

                                             catdesc  \
year problem                                           
2017 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
2018 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12      

### Merge the Datasets

In [14]:
msa_fips_crosswalk = pd.read_excel("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\list1_2023.xlsx",skiprows=2,dtype=str)
msa_fips_crosswalk.head()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,013,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,045,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,027,Central
3,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Callahan County,Texas,48,059,Outlying
4,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Jones County,Texas,48,253,Outlying


In [15]:
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS State Code"] + msa_fips_crosswalk["FIPS County Code"]

Drop NaN FIPS codes in preparation for merging.

In [16]:
msa_fips_crosswalk = msa_fips_crosswalk[msa_fips_crosswalk["FIPS Code"].notna()]
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS Code"].astype(int)
msa_fips_crosswalk["FIPS Code"]

0       46013
1       46045
2       53027
3       48059
4       48253
        ...  
1910     6101
1911     6115
1912     4027
1913    39119
1914    48505
Name: FIPS Code, Length: 1915, dtype: int64

In [17]:
crowd_data_msa = crowd_data.merge(msa_fips_crosswalk[["FIPS Code", "CBSA Code"]], how="left", left_on="fips_code", right_on="FIPS Code")
crowd_data_msa.head()


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code,year,FIPS Code,CBSA Code
0,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0,2017,27053.0,33460
1,2017-01-03,Palmetto Bay,FL,Palmetto Bay Police Station,0.0,demonstration,NaN,"Local pastors, family of Jamal Rollins","in remembrance of Jamal Rollins, against polic...",1.0,...,NaN,25.621772,-80.324775,Palmetto Bay,Miami-Dade County,FL,12086.0,2017,12086.0,33100
2,2017-01-04,Ypsilanti,MI,Eastern Michigan University,0.0,protest,NaN,"EMU's NAACP Chapter, EMU students","against racial incidents on campus, for pressu...",1.0,...,NaN,42.241150,-83.612994,Ypsilanti,Washtenaw County,MI,26161.0,2017,26161.0,11460
3,2017-01-04,Selfridge,ND,prairie near Cannonball and Missouri Rivers,0.0,protest; occupying land,NaN,protestors fighting the establishment of the D...,"for environmental rights, against Dakota Acces...",1.0,...,location is approximation based on descriptions,46.041389,-100.925130,Selfridge,Sioux County,ND,38085.0,2017,NaN,NaN
4,2017-01-04,New York,NY,Trump Tower,0.0,protest,NaN,Mexican politician against Trump's calls for c...,against Trump's campaign and upcoming presidency,1.0,...,NaN,40.712775,-74.005973,New York,NaN,NY,36061.0,2017,36061.0,35620


In [18]:
crowd_census_data = crowd_data_msa.merge(census_data, how="left", 
                                         left_on = ["year","CBSA Code"], right_on = ["year","METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA"])

In [19]:
crowd_census_data.head()

,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086PE,DP05_0087PE,DP05_0088PE,DP05_0089PE
0,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,6.7,0.0,0.1,2.7,0.1,2.6,NaN,2591001.0,49.0,51.0
1,2017-01-03,Palmetto Bay,FL,Palmetto Bay Police Station,0.0,demonstration,NaN,"Local pastors, family of Jamal Rollins","in remembrance of Jamal Rollins, against polic...",1.0,...,2.4,0.0,0.4,1.3,0.2,1.1,NaN,3924471.0,47.4,52.6
2,2017-01-04,Ypsilanti,MI,Eastern Michigan University,0.0,protest,NaN,"EMU's NAACP Chapter, EMU students","against racial incidents on campus, for pressu...",1.0,...,9.0,0.0,0.3,4.1,0.1,4.0,NaN,278978.0,49.2,50.8
3,2017-01-04,Selfridge,ND,prairie near Cannonball and Missouri Rivers,0.0,protest; occupying land,NaN,protestors fighting the establishment of the D...,"for environmental rights, against Dakota Acces...",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-04,New York,NY,Trump Tower,0.0,protest,NaN,Mexican politician against Trump's calls for c...,against Trump's campaign and upcoming presidency,1.0,...,11.3,0.0,0.7,1.7,0.2,1.5,NaN,13563604.0,47.2,52.8


#### Merge on Issue Salience

Some crowds are gathered for issues that fit into multiple categories, organized in alphabetical order. If this is the case, how should they be merged?

In [20]:
crowd_census_data["issues"] = crowd_census_data["issues"].apply(lambda x:np.array(x.split("; ")))

In [21]:
crowd_census_data["issues"]

0        [banking and finance, economy, energy, environ...
1                                               [policing]
2                                      [education, racism]
3                                    [energy, environment]
4                                             [presidency]
                               ...                        
16763                                    [foreign affairs]
16764                                   [policing, racism]
16765                                            [housing]
16766                                     [women's rights]
16767                                   [criminal justice]
Name: issues, Length: 16768, dtype: object

Perhaps solely the issue with the highest salience should be accounted for? Or, perhaps, the pcts should be added, representing the salience of anyone interested in at least one of the issues?

"The percentages can therefore be interpreted as the percentage of the electorate who answered the question and selected that category." -MIPDV2-Codebook p. 5

In [26]:
np.unique(np.concat(crowd_census_data["issues"]))

array(['animal rights', 'banking and finance', 'civil rights',
       'corruption', 'covid', 'criminal justice', 'democracy',
       'development', 'disability rights', 'drugs', 'economy',
       'education', 'energy', 'environment', 'foreign affairs',
       'free speech', 'guns', 'healthcare', 'housing', 'immigration',
       'indigenous peoples', 'judiciary', 'labor', 'legislative',
       'lgbtqia', 'military', 'patriotism', 'policing', 'presidency',
       'racism', 'religion', 'reproductive rights', 'science',
       'sexual violence', 'sports', 'taxes', 'transportation',
       "women's rights"], dtype='<U19')

All the issues in the CCC data. Each is to be mapped to a problem code in the MIPD data. This will be done by hand.

But why is `covid` there? This should be limited to the years 2017 and 18.

In [28]:
crowd_census_data_covid = crowd_census_data["issues"].apply(lambda x:"covid" in x)
crowd_census_data[crowd_census_data_covid]

,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086PE,DP05_0087PE,DP05_0088PE,DP05_0089PE
4693,2017-09-30,Denver,CO,Downtown Denver,0.0,rally,NaN,general protestors,Rally against court's decesion to reopen Rocky...,0.0,...,4.3,0.1,0.2,2.2,0.1,2.2,NaN,2043836.0,49.2,50.8
13424,2018-05-04,Atlanta,GA,CDC Headquarters,0.0,protest; rally,NaN,general protestors,Protesting CDC definition of lyme disease,0.0,...,6.1,0.1,0.4,2.2,0.1,2.1,NaN,4111190.0,47.1,52.9
15010,2018-07-06,Raymondville,TX,outside Willacy County courthouse,0.0,protest,NaN,ARISE,against ICE; against reopening immigrant deten...,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15070,2018-07-11,San Francisco,CA,outside Japanese Consulate,0.0,protest,NaN,general protestors,protesting reopening of Japanese nuclear power...,0.0,...,26.3,0.6,0.4,4.5,0.2,4.3,NaN,3230062.0,49.0,51.0


According to the data dictionary, "These are generated after data compilation by running a series of regular expressions over the claims description text" (p. 3). These claims all include the words "reopen" or "reopening"; these are false positives. As such, `covid` will manually be removed from these four so as to not impact findings.

In [39]:
crowd_census_data.loc[crowd_census_data_covid,"issues"] = crowd_census_data.loc[crowd_census_data_covid, "issues"].apply(lambda x:np.setdiff1d(x, ("covid")))
crowd_census_data.loc[crowd_census_data_covid,"issues"]

4693       [judiciary]
13424     [healthcare]
15010    [immigration]
15070         [energy]
Name: issues, dtype: object

In [43]:
crowd_census_data.loc[crowd_census_data["issues"].apply(lambda x:"science" in x)]

,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086PE,DP05_0087PE,DP05_0088PE,DP05_0089PE
1451,2017-02-19,Boston,MA,Copley Square,0.0,protest,NaN,"ClimateTruth.org, Union of Concerned Scientist...","pro-science, pro-environment, support addressi...",1.0,...,7.9,0.0,0.9,2.0,0.2,1.7,NaN,3462624.0,47.7,52.3
2297,2017-04-21,Hilo,HI,Hilo Bayfront’s soccer pavilion,0.0,march,NaN,general protestors,against political suppression of science,1.0,...,21.7,10.8,0.2,24.1,0.2,23.9,NaN,143064.0,49.5,50.5
2302,2017-04-22,Anchorage,AK,NaN,0.0,rally,NaN,general protestors,against political suppression of science,1.0,...,8.2,2.0,0.2,7.8,0.1,7.6,NaN,285990.0,51.8,48.2
2303,2017-04-22,Atka Island,AK,NaN,0.0,demonstration,NaN,general protestors,against political suppression of science,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2304,2017-04-22,Dillingham,AK,NaN,0.0,march,NaN,general protestors,against political suppression of science,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13186,2018-04-22,Rapid City,SD,Surbeck Center,0.0,rally,NaN,general protestors,for science,1.0,...,1.4,0.0,0.0,3.4,0.0,3.4,NaN,112012.0,50.1,49.9
13689,2018-05-19,Saint Paul,MN,Mears Park,0.0,march,NaN,general protestors,March for Science,1.0,...,6.6,0.0,0.2,3.0,0.1,2.9,NaN,2612672.0,49.0,51.0
13690,2018-05-19,St. Paul,MN,Mears Park,0.0,march,NaN,March for Science,March for Science,1.0,...,6.6,0.0,0.2,3.0,0.1,2.9,NaN,2612672.0,49.0,51.0
13721,2018-05-20,Eau Claire,WI,UW-Eau Claire,0.0,march,NaN,general protestors,March for Science,0.0,...,3.3,0.0,0.0,2.5,0.1,2.4,NaN,132402.0,50.1,49.9


Check again for unique issues:

In [40]:
np.unique(np.concat(crowd_census_data["issues"]))

array(['animal rights', 'banking and finance', 'civil rights',
       'corruption', 'criminal justice', 'democracy', 'development',
       'disability rights', 'drugs', 'economy', 'education', 'energy',
       'environment', 'foreign affairs', 'free speech', 'guns',
       'healthcare', 'housing', 'immigration', 'indigenous peoples',
       'judiciary', 'labor', 'legislative', 'lgbtqia', 'military',
       'patriotism', 'policing', 'presidency', 'racism', 'religion',
       'reproductive rights', 'science', 'sexual violence', 'sports',
       'taxes', 'transportation', "women's rights"], dtype='<U19')

In [41]:
issue_data[["problemname","problemdesc","problemissues","problemexamples","perc"]]

problemname  \
year problem                                     
2017 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
2018 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   

                                                    problemdesc  \
year problem                                                      
2017 1                 includes all mentions of economic issues   
     2                   includes all mentions of social policy   
     3        includes all mentions of civil and political r...   
     4         includes all mentions of crime and public safety   
     5            includes all mentions of budgets and spending   
     6        includes all mentions of foreign policy and ot...   
     7        includes all mentions of international economi...   
     8        includes all mentions of the environment, natu...   
     9        includes all mentions of morals, values, and r...   
     10       includes all mentions of political leaders, va...   
     11                   includes all mentions of youth issues   
     12                includes all mentions of minority groups   
     13       includes mentions of other problems or all pro...   
     14                        includes mentions of no problems   
     15       includes any mention that is related to don't ...   
2018 1                 includes all mentions of economic issues   
     2                   includes all mentions of social policy   
     3        includes all mentions of civil and political r...   
     4         includes all mentions of crime and public safety   
     5            includes all mentions of budgets and spending   
     6        includes all mentions of foreign policy and ot...   
     7        includes all mentions of international economi...   
     8        includes all mentions of the environment, natu...   
     9        includes all mentions of morals, values, and r...   
     10       includes all mentions of political leaders, va...   
     11                   includes all mentions of youth issues   
     12                includes all mentions of minority groups   
     13       includes mentions of other problems or all pro...   
     14                        includes mentions of no problems   
     15       includes any mention that is related to don't ...   

                                                  problemissues  \
year problem                                                      
2017 1        Economy (general), unemployment, inflation, gr...   
     2        Social policy (general), welfare, education, h...   
     3        Gender 

In [44]:
issue_code_dict = {'animal rights':14, 
                   'banking and finance':1, 
                   'civil rights':3,
                   'corruption':10, 
                   'criminal justice':3, 
                   'democracy':10,
                   'development':1, 
                   'disability rights':3, 
                   'drugs':4, 
                   'economy':1,
                   'education':2, 
                   'energy':8, 
                   'environment':8, 
                   'foreign affairs':6,
                   'free speech':3, 
                   'guns':4, 
                   'healthcare':2, 
                   'housing':2, 
                   'immigration':5,
                   'indigenous peoples':12, 
                   'judiciary':10, 
                   'labor':1, 
                   'legislative':10,
                   'lgbtqia':3, 
                   'military':6, 
                   'patriotism':9, 
                   'policing':3,
                   'presidency':10,
                   'racism':3, 
                   'religion':9, 
                   'reproductive rights':3, 
                   'science':10,
                   'sexual violence':4, 
                   'sports':13, 
                   'taxes':1, 
                   'transportation':2,
                   "women's rights":3
}